In [2]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

import pickle as pk
from PIL import Image
import random

PyTorch Version:  1.0.0
Torchvision Version:  0.2.1


In [3]:
# Detect if we have a GPU available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:',device)

batch_size = 32

device: cuda


In [4]:
from torch.utils.data import Dataset, DataLoader
def dataLoader():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # mean = [0.485, 0.456, 0.406]  std = [0.229, 0.224, 0.225]
    test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])
    Gallery_dataset = datasets.ImageFolder(root='../数据集/UCMerced_LandUse/data/test', transform=test_augs)
#     Gallery_dataset = datasets.ImageFolder(root='../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif', transform=test_augs)
    return Gallery_dataset
Gallery_dataset = dataLoader()
Gallery_loader = DataLoader(Gallery_dataset,
                           batch_size = 32, shuffle=False)

In [5]:
## 加载模型：resnet34_whurisi_remote_finetune_
pretrained_net = models.resnet34(pretrained=True)
# pretrained_net.fc = nn.Linear(512, 21)
# # pretrained_net.avgpool = nn.MaxPool2d(kernel_size=7, stride=1, padding=0)
# PATH = "./my_model/UcRemote_resnet34_fine-tune.pt"  # 默认使用平均池化
# # PATH = "./my_model/maxPooling_UcRemote_resnet34_fine-tune.pt"  # 使用最大池化
# pretrained_net.fc = nn.Sequential()
# pretrained_net.load_state_dict(torch.load(PATH))

In [6]:
dir(pretrained_net)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backend',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_make_layer',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'avgpool',
 'bn1',
 'buffers',
 'children',
 'conv1',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'fc',
 'float',
 'forward',
 'half',
 'inplanes',
 'layer1',

In [7]:
pretrained_net.avgpool = nn.Sequential()
pretrained_net.fc = nn.Sequential()

In [8]:
pretrained_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [9]:
pretrained_net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [51]:
# 读取一张图片并预处理
img_path = 'database/hotdog/train/hotdog/785.png'
def load_query_image(img_path):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    test_augs = transforms.Compose([
            transforms.Resize(size=256),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(),
            normalize
        ])
    query_image = datasets.folder.default_loader(img_path)
    query_image = test_augs(query_image)
    query_image = query_image.view(-1,3,224,224) # 3,224,224 是预设的 要输入给网络的
    return query_image
query_image = load_query_image(img_path)

In [52]:
query_image = query_image.to(device)
pretrained_net = pretrained_net.to(device)
X = query_image

In [73]:
import cv2
def gaussian_kernel_2d_opencv(kernel_size = 3,sigma = 0):
    kx = cv2.getGaussianKernel(kernel_size,sigma)
    ky = cv2.getGaussianKernel(kernel_size,sigma)
    return np.multiply(kx,np.transpose(ky))
def spoc_pooling(intput):
    kernel_size = intput.size(2)
    out = intput.cpu().detach().numpy()
    out = out * gaussian_kernel_2d_opencv(kernel_size)
    out = torch.from_numpy(out)
    out = out.to(device)
    pooling = nn.AvgPool2d(kernel_size=kernel_size, stride=1, padding=0)
    out = pooling(out)
    return out

In [74]:
# 只是为了去掉view那一步而已 然后加上spoc
def extractF_net(net,X):
    for name, layer in pretrained_net.named_children():
        X = layer(X)
    out = spoc_pooling(X)
    return out

In [75]:
out = extractF_net(pretrained_net, query_image)

In [76]:
out.shape

torch.Size([1, 512, 1, 1])

In [77]:
img2_path = 'database/hotdog/test/hotdog/1001.png' # hotdog
img3_path = 'database/hotdog/test/not-hotdog/1012.png' # not hotdog
img4_path = 'database/hotdog/test/not-hotdog/1065.png' # not hotdog 这个非常不像
img2 = load_query_image(img2_path)
img3 = load_query_image(img3_path)
img4 = load_query_image(img4_path)

In [78]:
img2 = img2.to(device)
output_test2 = extractF_net(pretrained_net,img2)
img3 = img3.to(device)
output_test3 = extractF_net(pretrained_net,img3)
img4 = img4.to(device)
output_test4 = extractF_net(pretrained_net,img4)

In [79]:
output_test2.shape, output_test3.shape, output_test4.shape

(torch.Size([1, 512, 1, 1]),
 torch.Size([1, 512, 1, 1]),
 torch.Size([1, 512, 1, 1]))

In [81]:
# 打平
output_test2 = output_test2.view(512)
output_test3 = output_test3.view(512)
output_test4 = output_test4.view(512)

output_test2 = output_test2.data.cpu()
output_test3 = output_test3.data.cpu()
output_test4 = output_test4.data.cpu()

In [83]:
# 相似度
out1 = (output_test2*output_test3)/(torch.norm(output_test3,p=2,dim=0, keepdim=True)*torch.norm(output_test2,p=2,dim=0, keepdim=True))
out2 = (output_test2*output_test4)/(torch.norm(output_test2,p=2,dim=0, keepdim=True)*torch.norm(output_test4,p=2,dim=0, keepdim=True))

In [85]:
out1.sum(), out2.sum()

(tensor(0.6094, dtype=torch.float64), tensor(0.3449, dtype=torch.float64))

## 高斯函数

In [19]:
import cv2
def gaussian_kernel_2d_opencv(kernel_size = 3,sigma = 0):
    kx = cv2.getGaussianKernel(kernel_size,sigma)
    ky = cv2.getGaussianKernel(kernel_size,sigma)
    return np.multiply(kx,np.transpose(ky))

In [35]:
func_W = gaussian_kernel_2d_opencv()

In [36]:
a = [[1,2,3],[4,5,6],[7,8,9]]
x = np.asarray(a)

In [37]:
x

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [40]:
func_W

array([[0.0625, 0.125 , 0.0625],
       [0.125 , 0.25  , 0.125 ],
       [0.0625, 0.125 , 0.0625]])

In [41]:
x_f = func_W * x

In [42]:
x_f

array([[0.0625, 0.25  , 0.1875],
       [0.5   , 1.25  , 0.75  ],
       [0.4375, 1.    , 0.5625]])